In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

## 01 Data pre-processing

### 01.01 Explore the dataset

```python
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print("Number of training examples: " + str(m_train))
print("Number of testing examples: " + str(m_test))
print("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print("train_x_orig shape: " + str(train_x_orig.shape))
print("train_y shape: " + str(train_y.shape))
print("test_x_orig shape: " + str(test_x_orig.shape))
print("test_y shape: " + str(test_y.shape))
```

### 01.02 Reshaping

```python
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T # -1 flatten the remaining dims
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T
```

### 01.03 Standardization

```python
train_x = train_x_flatten/255
test_x = test_x_flatten/255

print("train_x's shape: " + str(train_x.shape))
print("test_x's shape: " + str(test_x.shape))
```

## 02 Models

### 02.01 2-layer NN
Implementation of a two-layer neural network: LINEAR->RELU->LINEAR->SIGMOID.

#### 02.01.01 Constants

In [5]:
n_x = 12288     # num_px * num_px * 3
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)
learning_rate = 0.0075

#### 02.01.02 Model

In [2]:
def two_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Arguments:
    X: input data, of shape (n_x, number of examples)
    Y:- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims: dims of the layers (n_x, n_h, n_y)
    num_iterationsL num of iterations of the optimization loop
    learning_rate: learning rate of the gradient descent update rule
    print_cost: If True, this will print the cost every 100 iterations 
    
    Returns:
    parameters: dict containing W1, W2, b1, and b2
    """
    
    # np.random.seed(1)
    grads = {}
    costs = []                               # keep track of the cost
    m = X.shape[1]                           # number of examples
    (n_x, n_h, n_y) = layers_dims
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # gradient descent loop
    for i in range(0, num_iterations):

        # Forward prop: LINEAR -> RELU -> LINEAR -> SIGMOID
        A1, cache1 = linear_activation_forward(X, W1, b1, 'relu')
        A2, cache2 = linear_activation_forward(A1, W2, b2, 'sigmoid')
        
        # Compute cost
        cost = compute_cost(A2, Y)
        
        # Initializing back prop
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Back prop
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, 'sigmoid')
        dA0, dW1, db1 = linear_activation_backward(dA1, cache1, 'relu')
            
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
            
    return parameters, costs

In [3]:
def plot_costs(costs, learning_rate=0.0075):
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

#### 02.01.03 Compute cost
```python
parameters, costs = two_layer_model(train_x, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2, print_cost=False)

print("Cost after first iteration: " + str(costs[0]))

two_layer_model_test(two_layer_model)
```

#### 02.01.04 Training
```python
parameters, costs = two_layer_model(train_x, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)
plot_costs(costs, learning_rate)
```

#### 02.01.04 Predictions
```python
predictions_train = predict(train_x, train_y, parameters) # predict on train data
predictions_test = predict(test_x, test_y, parameters) # predict on test data
```

### 02.02 L-layer model
Builds a $L$-layer neural network of the ff. structure: [LINEAR -> RELU]$\times$(L-1) -> LINEAR -> SIGMOID.

#### 02.02.01 Constants

In [4]:
layers_dims = [12288, 20, 7, 5, 1] #  4-layer model

#### 02.02.02 Model

In [6]:
# GRADED FUNCTION: L_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Arguments:
    X: input data, of shape (n_x, number of examples)
    Y : true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims: list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate: learning rate of the gradient descent update rule
    num_iterations: num of iterations of the optimization loop
    print_cost: if True, it prints the cost every 100 steps
    
    Returns:
    parameters: params learnt by the model. They can then be used to predict.
    """

    # np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization
    parameters = initialize_parameters_deep(layers_dims)
    
    # Gradient descent loop
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
        
        # Compute cost
        cost = compute_cost(AL, Y)
        
        # Back prop
        grads = L_model_backward(AL, Y, caches)

        # Update parameters
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 iterations and for the last iteration
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

#### 02.02.03 Compute cost
```python
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 1, print_cost = False)

print("Cost after first iteration: " + str(costs[0]))

L_layer_model_test(L_layer_model)
```

#### 02.02.04 Training
```python
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)
```

#### 02.02.05 Predictions
```python
pred_train = predict(train_x, train_y, parameters)
pred_test = predict(test_x, test_y, parameters)
```